<a href="https://colab.research.google.com/github/pulavartivinay/Face_Mask_Detection/blob/stream/streamModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>